[mnist実験いろいろ](https://toukei-lab.com/mnist)

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling２D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold
import cv2
from PIL import Image
import os.path as osp
import os
from math import pi
import torch
import pickle
import scipy as sp
import pandas as pd
#os.environ['KMP_DUPLICATE_LIB_OK']='True'
HOME = osp.expanduser("~")
HOME

2023-01-16 07:53:42.346397: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'/home/yy/y.shinozaki'


---
(MNIST参考)[ https://toukei-lab.com/mnist ] 

In [18]:
# Kerasに付属の手書き数字画像データをダウンロード
#np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()
X_test = dataset_second_gauss_test_3_to_4_ver2["test_4_imag"]
X_train_base = dataset_train_4_imag_ver2

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
"""X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)"""

#正規化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
"""X_train /= 255
X_validation /= 255
X_test /= 255"""
X_train = sp.stats.zscore(X_train.reshape(48000,784),axis=1)
X_validation = sp.stats.zscore(X_validation.reshape(12000,784),axis=1)
X_test = sp.stats.zscore(X_test.reshape(10000,784),axis=1)
"""X_train = X_train.reshape(48000,28,28)
X_validation = X_validation.reshape(12000,28,28)
X_test = X_test.reshape(10000,28,28)"""
# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.003

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
"""model.add(Dense(32,input_shape=(28,28,1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))"""
model.add(Dense(n_hidden,activation='relu',input_shape=(n_features,)))
model.add(Dense(n_hidden,activation='relu'))
model.add(Dense(n_hidden,activation='relu'))
# 出力層を追加
#model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(X_train, y_train, epochs=20, batch_size=200, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     min_delta=0, patience=10, 
                                                         verbose=1)],
                validation_data=(X_validation, y_validation))

# Test dataで予測を実行。
#pred_test = model.predict(X_test)
#pred_test = (model.predict(X_test) > 0.5).astype("int32")
pred_test = np.argmax(model.predict(X_test),axis=1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

AxisError: axis 2 is out of bounds for array of dimension 2

---
(MNIST(pytorch))[ https://qiita.com/knyrc/items/0a0092b9903b97fb41b4 ]

---

In [247]:
# Kerasに付属の手書き数字画像データをダウンロード(リンクの練習用)
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#正規化

X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.01

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
model.add(Dense(n_hidden,activation='relu',input_shape=(n_features,)))
model.add(Dense(n_hidden,activation='relu'))
model.add(Dense(n_hidden,activation='relu'))

# 出力層を追加
model.add(Dense(10,activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(X_train, y_train, epochs=3000, batch_size=100, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     min_delta=0, patience=10, 
                                                         verbose=1)],
                validation_data=(X_validation, y_validation))

# Test dataで予測を実行。
pred_test = model.predict_classes(X_test)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

ValueError: Unknown activation function: step

In [3]:
cd MNIST_pickle/second_MNIST/

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle/second_MNIST


In [4]:
#　訓練データのロード

from sklearn import preprocessing

# 0
with open("second_gauss_train_0_real_ver2.pkl","rb") as f:
    dataset_train_0_real_ver2 = pickle.load(f)
# 1
with open("second_gauss_train_1_real_ver2.pkl","rb") as f:
    dataset_train_1_real_ver2 = pickle.load(f)
with open("second_gauss_train_1_imag_ver2.pkl","rb") as f:
    dataset_train_1_imag_ver2 = pickle.load(f)
# 2
with open("second_gauss_train_2_real_ver2.pkl","rb") as f:
    dataset_train_2_real_ver2 = pickle.load(f)
with open("second_gauss_train_2_imag_ver2.pkl","rb") as f:
    dataset_train_2_imag_ver2 = pickle.load(f)
# 3
with open("second_gauss_train_3_real_ver2.pkl","rb") as f:
    dataset_train_3_real_ver2 = pickle.load(f)
with open("second_gauss_train_3_imag_ver2.pkl","rb") as f:
    dataset_train_3_imag_ver2 = pickle.load(f)
# 4
with open("second_gauss_train_4_real_ver2.pkl","rb") as f:
    dataset_train_4_real_ver2 = pickle.load(f)
with open("second_gauss_train_4_imag_ver2.pkl","rb") as f:
    dataset_train_4_imag_ver2 = pickle.load(f)

#　テストデータのロード
with open("second_gauss_test_0_to_2_ver2.pkl","rb") as f:
    dataset_second_gauss_test_0_to_2_ver2 = pickle.load(f)
with open("second_gauss_test_3_to_4_ver2.pkl","rb") as f:
    dataset_second_gauss_test_3_to_4_ver2 = pickle.load(f)

#　正解(ラベル)データのロード
with open("raw_mnist.pkl","rb") as f:
     dataset_raw = pickle.load(f)

In [26]:
#　機械学習
    
#X_train_basement = np.vstack((dataset_train_0_to_2["second_gauss_train_0_to_2"],dataset_train_3_to_4["second_gauss_train_3_to_4"]))
#for i in range(5,6):
# Kerasに付属の手書き数字画像データをダウンロード
#labels_train_base = dataset_raw["y_train"]

#X_train_base = np.vstack((dataset_train_0_to_2["second_gauss_train_0_to_2"],dataset_train_3_to_4["second_gauss_train_3_to_4"]))
#X_test = dataset_second_gauss_test_0_to_2_ver2["test_4_imag"]
X_test = dataset_second_gauss_test_3_to_4_ver2["test_3_real"]
labels_train_base = dataset_raw["y_train"]
labels_test = dataset_raw["y_test"]

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(dataset_train_3_real_ver2,labels_train_base,test_size = 0.2) 

#正規化 (行ごとの正規化)
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train = sp.stats.zscore(X_train.reshape(48000,784),axis=1)
X_validation = sp.stats.zscore(X_validation.reshape(12000,784),axis=1)
X_test = sp.stats.zscore(X_test.reshape(10000,784),axis=1)
"""
X_train = preprocessing.minmax_scale(X_train.reshape(48000,784),axis=1)
X_validation = preprocessing.minmax_scale(X_validation.reshape(12000,784),axis=1)
X_test = preprocessing.minmax_scale(X_test.reshape(10000,784),axis=1)
"""
"""y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    
"""
#　さらにreshape
X_train = X_train.reshape(48000,28,28)
X_validation = X_validation.reshape(12000,28,28)
X_test = X_test.reshape(10000,28,28)
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    
    
# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.003

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
model.add(Dense(32,input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 出力層を追加
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
                  loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(X_train, y_train, epochs=20, batch_size=200, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                       min_delta=0, patience=10, 
                                                          verbose=1)],
                validation_data=(X_validation, y_validation))

hist_df = pd.DataFrame(log.history)
hist_df.to_csv("history_3_real.csv")

# モデルを保存
model.save_weights(HOME+"/研究/MNIST/MNIST_models/checkpoint_3_real")
# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

Epoch 1/20
240/240 [==============================] - 20s 81ms/step - loss: 0.2697 - mae: 0.0247 - accuracy: 0.9155 - val_loss: 0.1456 - val_mae: 0.0151 - val_accuracy: 0.9535
Epoch 2/20
240/240 [==============================] - 18s 75ms/step - loss: 0.1452 - mae: 0.0138 - accuracy: 0.9549 - val_loss: 0.1393 - val_mae: 0.0136 - val_accuracy: 0.9563
Epoch 3/20
240/240 [==============================] - 21s 87ms/step - loss: 0.1162 - mae: 0.0112 - accuracy: 0.9631 - val_loss: 0.1162 - val_mae: 0.0100 - val_accuracy: 0.9634
Epoch 4/20
240/240 [==============================] - 21s 87ms/step - loss: 0.1002 - mae: 0.0097 - accuracy: 0.9679 - val_loss: 0.1329 - val_mae: 0.0101 - val_accuracy: 0.9611
Epoch 5/20
240/240 [==============================] - 21s 87ms/step - loss: 0.0870 - mae: 0.0086 - accuracy: 0.9714 - val_loss: 0.1196 - val_mae: 0.0100 - val_accuracy: 0.9641
Epoch 6/20
240/240 [==============================] - 22s 91ms/step - loss: 0.0800 - mae: 0.0080 - accuracy: 0.9734 - va

In [15]:
read_csv = pd.read_csv('history_0_real.csv', sep=',')

In [99]:
%matplotlib inline
import matplotlib.pyplot as plt

def plot_history(history, 
                save_graph_img_path, 
                fig_size_width, 
                fig_size_height, 
                lim_font_size):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
   
    epochs = range(len(acc))

    # グラフ表示
    plt.figure(figsize=(fig_size_width, fig_size_height))
    
    
    #plt.rcParams['font.family'] = 'Times New Roman'
    plt.rcParams['font.size'] = 50  # 全体のフォント
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'
    #plt.subplot(121)
    plt.grid()
    plt.axis([0,20, 0.9,1])
   
    plt.xticks(np.arange(0,22,4))
    plt.xlim([-0.5,22])
    plt.ylim([0.92,1.0])
    
    
    # plot accuracy values
    plt.plot(epochs, acc, color = "blue", linestyle = "solid", label = 'test acc', linewidth=5)
    plt.plot(epochs, val_acc, color = "green", linestyle = "solid", label= 'valid acc', linewidth=5)
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.legend()
    
    plt.savefig(save_graph_img_path)
    plt.tight_layout()
    plt.show()
    plt.close() # バッファ解放

(MNISTモデルの使い方)[　https://qiita.com/zunda_pixel/items/6178855310302efabfcc　]

In [151]:
# 学習率
rate       = 0.003

# テストデータ準備
X_test = dataset_second_gauss_test_3_to_4_ver2["test_3_real"]
X_test = X_test.astype('float32')
X_test = sp.stats.zscore(X_test.reshape(10000,784),axis=1)
X_test = sp.stats.zscore(X_test.reshape(10000,28,28))
                         
# Sequentialクラスを使ってモデルを準備する
model = Sequential()
# 隠れ層を追加
model.add(Dense(32,input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 出力層を追加
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),loss='categorical_crossentropy', metrics=['mae', 'accuracy'])
model.summary()
model.load_weights(HOME+"/研究/MNIST/MNIST_models/checkpoint_3_real")

pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

Model: "sequential_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_78 (Dense)            (None, 28, 28, 32)        64        
                                                                 
 conv2d_39 (Conv2D)          (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 13, 13, 32)       0         
 g2D)                                                            
                                                                 
 dropout_39 (Dropout)        (None, 13, 13, 32)        0         
                                                                 
 flatten_39 (Flatten)        (None, 5408)              0         
                                                                 
 dense_79 (Dense)            (None, 10)                54090     
                                                     

(参考)[ https://qiita.com/fukuit/items/b3fa460577a0ea139c88 ]

In [34]:
#　普通のMINIST
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()


# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
#X_train = X_train.reshape(-1,784)
#X_validation = X_validation.reshape(-1,784)
#X_test = X_test.reshape(-1,784)

#正規化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)


# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.005

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
#model.add(Dense(1,kernel_regularizer=keras.regularizers.l2(rate),input_shape=(28, 28, 1)))
#model.add(Conv2D(1, (3, 3), activation='relu'))
model.add(Conv2D(64,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
# 出力層を追加
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['categorical_crossentropy', 'accuracy'])

# Early stoppingを適用してフィッティング
log1 = model.fit(X_train, y_train, epochs=20, batch_size=100, verbose=True,
                #callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                  #                                min_delta=0, patience=10, 
                    #                                     verbose=1)],
                validation_data=(X_validation, y_validation))

# モデルを保存
model.save(HOME+"/研究/MNIST/second_MNIST/models/second_mnist_raw_model")

# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

Epoch 1/20
480/480 [==============================] - 18s 37ms/step - loss: 0.1949 - categorical_crossentropy: 0.1949 - accuracy: 0.9399 - val_loss: 0.0622 - val_categorical_crossentropy: 0.0622 - val_accuracy: 0.9811
Epoch 2/20
480/480 [==============================] - 26s 55ms/step - loss: 0.0721 - categorical_crossentropy: 0.0721 - accuracy: 0.9778 - val_loss: 0.0474 - val_categorical_crossentropy: 0.0474 - val_accuracy: 0.9857
Epoch 3/20
480/480 [==============================] - 22s 46ms/step - loss: 0.0593 - categorical_crossentropy: 0.0593 - accuracy: 0.9812 - val_loss: 0.0354 - val_categorical_crossentropy: 0.0354 - val_accuracy: 0.9883
Epoch 4/20
480/480 [==============================] - 20s 42ms/step - loss: 0.0527 - categorical_crossentropy: 0.0527 - accuracy: 0.9839 - val_loss: 0.0448 - val_categorical_crossentropy: 0.0448 - val_accuracy: 0.9852
Epoch 5/20
480/480 [==============================] - 22s 46ms/step - loss: 0.0462 - categorical_crossentropy: 0.0462 - accuracy

INFO:tensorflow:Assets written to: /home/yy/y.shinozaki/研究/MNIST/second_MNIST/models/second_mnist_raw_model/assets


INFO:tensorflow:Assets written to: /home/yy/y.shinozaki/研究/MNIST/second_MNIST/models/second_mnist_raw_model/assets


313/313 [==============================] - 1s 5ms/step
9914/10000 correct (99.140%)


In [28]:
#　普通のMINIST(K-fold cross validationあり)
np.random.seed(0)
#(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()


# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
(X_train,y_train), (X_test, y_test) = mnist.load_data()

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
#X_train = X_train.reshape(-1,784)
#X_validation = X_validation.reshape(-1,784)
#X_test = X_test.reshape(-1,784)

#正規化
X_train = X_train.astype('float32')
#X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
#X_validation /= 255
X_test /= 255

# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
#y_train = to_categorical(labels_train)
#y_validation = to_categorical(labels_validation)
#y_test = to_categorical(labels_test)


# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.005

# X-fold cross validationを定義
kfold = StratifiedKFold(n_splits=5, shuffle=True)
cvscores = []

for train, test in kfold.split(X_train, y_train):
    
    # Sequentialクラスを使ってモデルを準備する
    model = Sequential()

    # 隠れ層を追加
    model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))
    # 出力層を追加
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    # TensorFlowのモデルを構築
    model.compile(optimizer=tf.optimizers.Adam(rate),
                  loss='categorical_crossentropy', metrics=['categorical_crossentropy', 'accuracy'])

    # Early stoppingを適用してフィッティング
    log1 = model.fit(X_train[train], keras.utils.to_categorical(y_train[train],10), epochs=1, batch_size=100, verbose=1)

    # Test dataで予測を実行。
    # Evaluate
    scores = model.evaluate(X_train[test], keras.utils.to_categorical(y_train[test], 10), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    """
    pred_test = np.argmax(model.predict(X_test), axis=-1)

    validation = (pred_test == labels_test)
    size       = validation.size
    size
    correct    = np.count_nonzero(validation)
    print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")
    """
    
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

480/480 [==============================] - 36s 75ms/step - loss: 0.1958 - categorical_crossentropy: 0.1958 - accuracy: 0.9418
categorical_crossentropy: 11.49%
480/480 [==============================] - 38s 78ms/step - loss: 0.1989 - categorical_crossentropy: 0.1989 - accuracy: 0.9394
categorical_crossentropy: 8.33%
480/480 [==============================] - 36s 74ms/step - loss: 0.2002 - categorical_crossentropy: 0.2002 - accuracy: 0.9407
categorical_crossentropy: 9.22%
480/480 [==============================] - 32s 67ms/step - loss: 0.2044 - categorical_crossentropy: 0.2044 - accuracy: 0.9395
categorical_crossentropy: 10.30%
480/480 [==============================] - 37s 76ms/step - loss: 0.2010 - categorical_crossentropy: 0.2010 - accuracy: 0.9406
categorical_crossentropy: 9.54%
9.78% (+/- 1.06%)


In [12]:
pd.DataFrame(log1.history).to_csv(HOME+"/研究/MNIST/second_MNIST/second_mnist_raw_log.csv",index=False)
a = pd.read_csv(HOME+"/研究/MNIST/second_MNIST/second_mnist_raw_log.csv")
a
# 9830/10000 correct (98.300%)
#9847/10000 correct (98.470%)

,loss,mse,accuracy,val_loss,val_mse,val_accuracy
0,0.202244,0.009082,0.939500,0.093253,0.004388,0.970833
1,0.089231,0.004150,0.973000,0.070742,0.003324,0.978500
2,0.072009,0.003373,0.978083,0.069125,0.003337,0.978500
3,0.058736,0.002866,0.980812,0.058697,0.002735,0.982667
4,0.051509,0.002512,0.983583,0.057692,0.002794,0.981000
5,0.045185,0.002187,0.985729,0.053284,0.002510,0.982750
6,0.041567,0.002119,0.985979,0.052448,0.002493,0.983833
7,0.040123,0.002018,0.986750,0.061745,0.002859,0.981583
8,0.033918,0.001737,0.988437,0.059825,0.002729,0.982833
9,0.029331,0.001506,0.989937,0.068490,0.003018,0.980833


In [15]:
cd MNIST_pickle/second_MNIST

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle/second_MNIST


In [201]:
# 9層のバージョン
"""from sklearn import preprocessing
import pickle

#　訓練データのロード
# 0
with open("second_gauss_train_0_real_ver2.pkl","rb") as f:
    dataset_train_0_real_ver2 = pickle.load(f)
# 1
with open("second_gauss_train_1_real_ver2.pkl","rb") as f:
    dataset_train_1_real_ver2 = pickle.load(f)
with open("second_gauss_train_1_imag_ver2.pkl","rb") as f:
    dataset_train_1_imag_ver2 = pickle.load(f)
# 2
with open("second_gauss_train_2_real_ver2.pkl","rb") as f:
    dataset_train_2_real_ver2 = pickle.load(f)
with open("second_gauss_train_2_imag_ver2.pkl","rb") as f:
    dataset_train_2_imag_ver2 = pickle.load(f)
# 3
with open("second_gauss_train_3_real_ver2.pkl","rb") as f:
    dataset_train_3_real_ver2 = pickle.load(f)
with open("second_gauss_train_3_imag_ver2.pkl","rb") as f:
    dataset_train_3_imag_ver2 = pickle.load(f)
# 4
with open("second_gauss_train_4_real_ver2.pkl","rb") as f:
    dataset_train_4_real_ver2 = pickle.load(f)
with open("second_gauss_train_4_imag_ver2.pkl","rb") as f:
    dataset_train_4_imag_ver2 = pickle.load(f)

#　テストデータのロード
with open("second_gauss_test_0_to_2_ver2.pkl","rb") as f:
    dataset_second_gauss_test_0_to_2_ver2 = pickle.load(f)
with open("second_gauss_test_3_to_4_ver2.pkl","rb") as f:
    dataset_second_gauss_test_3_to_4_ver2 = pickle.load(f)

#　正解(ラベル)データのロード
with open("raw_mnist.pkl","rb") as f:
     dataset_raw = pickle.load(f)

###　訓練データ
#　reshapeと型変換
dataset_train_0_real_ver2 = dataset_train_0_real_ver2.astype('float32').reshape(60000,784)
dataset_train_1_real_ver2 = dataset_train_1_real_ver2.astype('float32').reshape(60000,784)
dataset_train_1_imag_ver2 = dataset_train_1_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_2_real_ver2 = dataset_train_2_real_ver2.astype('float32').reshape(60000,784)
dataset_train_2_imag_ver2 = dataset_train_2_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_3_real_ver2 = dataset_train_3_real_ver2.astype('float32').reshape(60000,784)
dataset_train_3_imag_ver2 = dataset_train_3_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_4_real_ver2 = dataset_train_4_real_ver2.astype('float32').reshape(60000,784)
dataset_train_4_imag_ver2 = dataset_train_4_imag_ver2.astype('float32').reshape(60000,784)
#　正規化
dataset_train_0_real_ver2 = sp.stats.zscore(dataset_train_0_real_ver2,axis=None)
dataset_train_1_real_ver2 = sp.stats.zscore(dataset_train_1_real_ver2,axis=None)
dataset_train_1_imag_ver2 = sp.stats.zscore(dataset_train_1_imag_ver2,axis=None)
dataset_train_2_real_ver2 = sp.stats.zscore(dataset_train_2_real_ver2,axis=None)
dataset_train_2_imag_ver2 = sp.stats.zscore(dataset_train_2_imag_ver2,axis=None)
dataset_train_3_real_ver2 = sp.stats.zscore(dataset_train_3_real_ver2,axis=None)
dataset_train_3_imag_ver2 = sp.stats.zscore(dataset_train_3_imag_ver2,axis=None)
dataset_train_4_real_ver2 = sp.stats.zscore(dataset_train_4_real_ver2,axis=None)
dataset_train_4_imag_ver2 = sp.stats.zscore(dataset_train_4_imag_ver2,axis=None)
X_train_base = np.stack((dataset_train_0_real_ver2.reshape(60000,28,28), dataset_train_1_real_ver2.reshape(60000,28,28),
                         dataset_train_1_imag_ver2.reshape(60000,28,28), dataset_train_2_real_ver2.reshape(60000,28,28), 
                         dataset_train_2_imag_ver2.reshape(60000,28,28), dataset_train_3_real_ver2.reshape(60000,28,28), 
                         dataset_train_3_imag_ver2.reshape(60000,28,28),dataset_train_4_real_ver2.reshape(60000,28,28),
                         dataset_train_4_imag_ver2.reshape(60000,28,28)),axis=3)
"""
for i in range(2*Q+1):
    second_gauss_train[i] = sp.stats.zscore(second_gauss_train[i],axis=None)
    second_gauss_test[i] = sp.stats.zscore(second_gauss_test[i],axis=None)
    
X_train_base =  np.stack((second_gauss_train[0].reshape(60000,28,28),second_gauss_train[1].reshape(60000,28,28),
                          second_gauss_train[2].reshape(60000,28,28),second_gauss_train[3].reshape(60000,28,28),
                          second_gauss_train[4].reshape(60000,28,28),second_gauss_train[5].reshape(60000,28,28),
                          second_gauss_train[6].reshape(60000,28,28),second_gauss_train[7].reshape(60000,28,28),
                          second_gauss_train[8].reshape(60000,28,28)),axis=3)
                   
X_test = np.stack((second_gauss_test[0].reshape(10000,28,28),second_gauss_test[1].reshape(10000,28,28),
                   second_gauss_test[2].reshape(10000,28,28),second_gauss_test[3].reshape(10000,28,28),
                   second_gauss_test[4].reshape(10000,28,28),second_gauss_test[5].reshape(10000,28,28),
                   second_gauss_test[6].reshape(10000,28,28),second_gauss_test[7].reshape(10000,28,28),
                   second_gauss_test[8].reshape(10000,28,28)),axis=3)


"""###
#　テストデータのロード
with open("second_gauss_test_0_to_2_ver2.pkl","rb") as f:
    dataset_second_gauss_test_0_to_2_ver2 = pickle.load(f)
with open("second_gauss_test_3_to_4_ver2.pkl","rb") as f:
    dataset_second_gauss_test_3_to_4_ver2 = pickle.load(f)

# reshapeと型変換
test_0_real = dataset_second_gauss_test_0_to_2_ver2["test_0_real"].astype('float32').reshape(10000,784)
test_1_real = dataset_second_gauss_test_0_to_2_ver2["test_1_real"].astype('float32').reshape(10000,784)
test_1_imag = dataset_second_gauss_test_0_to_2_ver2["test_1_imag"].astype('float32').reshape(10000,784)
test_2_real = dataset_second_gauss_test_0_to_2_ver2["test_2_real"].astype('float32').reshape(10000,784)
test_2_imag = dataset_second_gauss_test_0_to_2_ver2["test_2_imag"].astype('float32').reshape(10000,784)
test_3_real = dataset_second_gauss_test_3_to_4_ver2["test_3_real"].astype('float32').reshape(10000,784)
test_3_imag = dataset_second_gauss_test_3_to_4_ver2["test_3_imag"].astype('float32').reshape(10000,784)
test_4_real = dataset_second_gauss_test_3_to_4_ver2["test_4_real"].astype('float32').reshape(10000,784)
test_4_imag = dataset_second_gauss_test_3_to_4_ver2["test_4_imag"].astype('float32').reshape(10000,784)
# 行ごとの正規化
test_0_real = sp.stats.zscore(test_0_real,axis=None)
test_1_real = sp.stats.zscore(test_1_real,axis=None)
test_1_imag = sp.stats.zscore(test_1_imag,axis=None)
test_2_real = sp.stats.zscore(test_2_real,axis=None)
test_2_imag = sp.stats.zscore(test_2_imag,axis=None)
test_3_real = sp.stats.zscore(test_3_real,axis=None)
test_3_imag = sp.stats.zscore(test_3_imag,axis=None)
test_4_real = sp.stats.zscore(test_4_real,axis=None)
test_4_imag = sp.stats.zscore(test_4_imag,axis=None)
X_test = np.stack((test_0_real.reshape(10000,28,28), test_1_real.reshape(10000,28,28), test_1_imag.reshape(10000,28,28), 
                   test_2_real.reshape(10000,28,28), test_2_imag.reshape(10000,28,28), test_3_real.reshape(10000,28,28), 
                   test_3_imag.reshape(10000,28,28), test_4_real.reshape(10000,28,28),test_4_imag.reshape(10000,28,28)),axis=3)

"""
#　正解(ラベル)データのロード
with open("raw_mnist.pkl","rb") as f:
     dataset_raw = pickle.load(f)

In [171]:
X_train_base.shape

(9, 60000, 784)

In [15]:
# 5層のバージョン(Qごとの絶対値)
from sklearn import preprocessing
import pickle

#　訓練データのロード
# 0
with open("second_gauss_train_0_real_ver2.pkl","rb") as f:
    dataset_train_0_real_ver2 = pickle.load(f)
# 1
with open("second_gauss_train_1_real_ver2.pkl","rb") as f:
    dataset_train_1_real_ver2 = pickle.load(f)
with open("second_gauss_train_1_imag_ver2.pkl","rb") as f:
    dataset_train_1_imag_ver2 = pickle.load(f)
# 2
with open("second_gauss_train_2_real_ver2.pkl","rb") as f:
    dataset_train_2_real_ver2 = pickle.load(f)
with open("second_gauss_train_2_imag_ver2.pkl","rb") as f:
    dataset_train_2_imag_ver2 = pickle.load(f)
# 3
with open("second_gauss_train_3_real_ver2.pkl","rb") as f:
    dataset_train_3_real_ver2 = pickle.load(f)
with open("second_gauss_train_3_imag_ver2.pkl","rb") as f:
    dataset_train_3_imag_ver2 = pickle.load(f)
# 4
with open("second_gauss_train_4_real_ver2.pkl","rb") as f:
    dataset_train_4_real_ver2 = pickle.load(f)
with open("second_gauss_train_4_imag_ver2.pkl","rb") as f:
    dataset_train_4_imag_ver2 = pickle.load(f)

#　テストデータのロード
with open("second_gauss_test_0_to_2_ver2.pkl","rb") as f:
    dataset_second_gauss_test_0_to_2_ver2 = pickle.load(f)
with open("second_gauss_test_3_to_4_ver2.pkl","rb") as f:
    dataset_second_gauss_test_3_to_4_ver2 = pickle.load(f)

#　正解(ラベル)データのロード
with open("raw_mnist.pkl","rb") as f:
     dataset_raw = pickle.load(f)

###　訓練データ
#　reshapeと型変換
dataset_train_0_real_ver2 = dataset_train_0_real_ver2.astype('float32').reshape(60000,784)
dataset_train_1_real_ver2 = dataset_train_1_real_ver2.astype('float32').reshape(60000,784)
dataset_train_1_imag_ver2 = dataset_train_1_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_2_real_ver2 = dataset_train_2_real_ver2.astype('float32').reshape(60000,784)
dataset_train_2_imag_ver2 = dataset_train_2_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_3_real_ver2 = dataset_train_3_real_ver2.astype('float32').reshape(60000,784)
dataset_train_3_imag_ver2 = dataset_train_3_imag_ver2.astype('float32').reshape(60000,784)
dataset_train_4_real_ver2 = dataset_train_4_real_ver2.astype('float32').reshape(60000,784)
dataset_train_4_imag_ver2 = dataset_train_4_imag_ver2.astype('float32').reshape(60000,784)

#　正規化
dataset_train_0 = sp.stats.zscore(dataset_train_0_real_ver2,axis=None)
dataset_train_1 = sp.stats.zscore((dataset_train_1_real_ver2**2 + dataset_train_1_imag_ver2**2)**0.5,axis=None)
dataset_train_2 = sp.stats.zscore((dataset_train_2_real_ver2**2 + dataset_train_2_imag_ver2**2)**0.5,axis=None)
dataset_train_3 = sp.stats.zscore((dataset_train_3_real_ver2**2 + dataset_train_3_imag_ver2**2)**0.5,axis=None)
dataset_train_4 = sp.stats.zscore((dataset_train_4_real_ver2**2 + dataset_train_4_imag_ver2**2)**0.5,axis=None)


X_train_base = np.stack((dataset_train_0.reshape(60000,28,28), dataset_train_1.reshape(60000,28,28),
                         dataset_train_2.reshape(60000,28,28), dataset_train_3.reshape(60000,28,28), 
                         dataset_train_4.reshape(60000,28,28)),axis=3)

###
#　テストデータのロード
with open("second_gauss_test_0_to_2_ver2.pkl","rb") as f:
    dataset_second_gauss_test_0_to_2_ver2 = pickle.load(f)
with open("second_gauss_test_3_to_4_ver2.pkl","rb") as f:
    dataset_second_gauss_test_3_to_4_ver2 = pickle.load(f)

# reshapeと型変換
test_0_real = dataset_second_gauss_test_0_to_2_ver2["test_0_real"].astype('float32').reshape(10000,784)
test_1_real = dataset_second_gauss_test_0_to_2_ver2["test_1_real"].astype('float32').reshape(10000,784)
test_1_imag = dataset_second_gauss_test_0_to_2_ver2["test_1_imag"].astype('float32').reshape(10000,784)
test_2_real = dataset_second_gauss_test_0_to_2_ver2["test_2_real"].astype('float32').reshape(10000,784)
test_2_imag = dataset_second_gauss_test_0_to_2_ver2["test_2_imag"].astype('float32').reshape(10000,784)
test_3_real = dataset_second_gauss_test_3_to_4_ver2["test_3_real"].astype('float32').reshape(10000,784)
test_3_imag = dataset_second_gauss_test_3_to_4_ver2["test_3_imag"].astype('float32').reshape(10000,784)
test_4_real = dataset_second_gauss_test_3_to_4_ver2["test_4_real"].astype('float32').reshape(10000,784)
test_4_imag = dataset_second_gauss_test_3_to_4_ver2["test_4_imag"].astype('float32').reshape(10000,784)
# 行ごとの正規化
dataset_test_0 = sp.stats.zscore(test_0_real,axis=None)
dataset_test_1 = sp.stats.zscore((test_1_real**2 + test_1_imag**2)**0.5,axis=None)
dataset_test_2 = sp.stats.zscore((test_2_real**2 + test_2_imag**2)**0.5,axis=None)
dataset_test_3 = sp.stats.zscore((test_3_real**2 + test_3_imag**2)**0.5,axis=None)
dataset_test_4 = sp.stats.zscore((test_4_real**2 + test_4_imag**2)**0.5,axis=None)
X_test = np.stack((dataset_test_0.reshape(10000,28,28), dataset_test_1.reshape(10000,28,28), dataset_test_2.reshape(10000,28,28), 
                   dataset_test_3.reshape(10000,28,28), dataset_test_4.reshape(10000,28,28)),axis=3)

#　正解(ラベル)データのロード
with open("raw_mnist.pkl","rb") as f:
     dataset_raw = pickle.load(f)

In [290]:
from tensorflow.keras.layers import Flatten

# MNIST 複数チャンネル用
# Kerasに付属の手書き数字画像データをダウンロード
labels_train_base = dataset_raw["y_train"]
labels_test = dataset_raw["y_test"]
#X_train_base = X_train.transpose(0,2,3,1)

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2) 
# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き

#X_test = X_test.transpose(3,1,2,0)
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    

print("X_train:",X_train.shape)
print("X_validation:",X_validation.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_validation:",y_validation.shape)
print("y_test:",y_test.shape)

y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

bias_init = 0.1

# 学習率
rate       = 0.005

# Sequentialクラスを使ってモデルを準備する
model = Sequential()

# 隠れ層を追加
#model.add(Dense(32,kernel_regularizer=keras.regularizers.l2(rate),input_shape=(28, 28, 9)))
#model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 9)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())

# 出力層を追加
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['categorical_crossentropy', 'accuracy'])

# Early stoppingを適用してフィッティング
log2 = model.fit(X_train, y_train, batch_size=100 ,epochs=20, verbose=True,
                #callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                  #                                   min_delta=0, patience=10, 
                    #                                     verbose=1)],
                validation_data=(X_validation, y_validation))

#model.save(HOME+"/研究/MNIST/second_MNIST/models/second_mnist_abs_model")

# Test dataで予測を実行。
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

X_train: (48000, 28, 28, 9)
X_validation: (12000, 28, 28, 9)
X_test: (10000, 28, 28, 9)
y_train: (48000, 10)
y_validation: (12000, 10)
y_test: (10000, 10)
Epoch 1/20
480/480 [==============================] - 37s 76ms/step - loss: 0.1841 - categorical_crossentropy: 0.1841 - accuracy: 0.9485 - val_loss: 0.0789 - val_categorical_crossentropy: 0.0789 - val_accuracy: 0.9758
Epoch 2/20
480/480 [==============================] - 34s 71ms/step - loss: 0.0920 - categorical_crossentropy: 0.0920 - accuracy: 0.9736 - val_loss: 0.0724 - val_categorical_crossentropy: 0.0724 - val_accuracy: 0.9807
Epoch 3/20
424/480 [=========================>....] - ETA: 3s - loss: 0.0794 - categorical_crossentropy: 0.0794 - accuracy: 0.9780

KeyboardInterrupt: 

In [33]:
pd.DataFrame(log2.history).to_csv(HOME+"/研究/MNIST/second_MNIST/second_mnist_log.csv",index=False)

In [13]:
a = pd.read_csv(HOME+"/研究/MNIST/second_MNIST/second_mnist_log.csv")
a
# second_mnist_9layer: 9875/10000 correct (98.750%)
# second_mnist_5layer: 

,loss,mse,accuracy,val_loss,val_mse,val_accuracy
0,0.144794,0.006325,0.957542,0.074043,0.003142,0.979750
1,0.055256,0.002327,0.984875,0.062944,0.002770,0.981000
2,0.036789,0.001504,0.990104,0.066210,0.002828,0.981750
3,0.028665,0.001153,0.992583,0.059893,0.002462,0.983917
4,0.022400,0.000837,0.994646,0.056970,0.002332,0.984750
5,0.018792,0.000657,0.996042,0.057450,0.002295,0.985417
6,0.014657,0.000466,0.996979,0.063358,0.002313,0.985500
7,0.013249,0.000429,0.997312,0.062051,0.002241,0.985833
8,0.015703,0.000585,0.996229,0.065835,0.002458,0.984167
9,0.013498,0.000451,0.997187,0.068895,0.002531,0.984417


In [183]:
#MNIST複数チャンネル用(Flatニューラルネットワーク)
from tensorflow.keras.layers import Flatten

# MNIST 複数チャンネル用
# Kerasに付属の手書き数字画像データをダウンロード
labels_train_base = dataset_raw["y_train"]

#X_train_base = dataset_train_0_to_2["second_gauss_train_0_to_2"][2]
#X_train_base = np.vstack((dataset_train_0_to_2["second_gauss_train_0_to_2"],dataset_train_3_to_4["second_gauss_train_3_to_4"]))
#X_train_base = X_train_base.transpose(1,0,2)


#X_test = dataset_test["second_gauss_test"]
#X_test = X_test.reshape(9,10000,784)
#X_test = X_test.transpose(1,0,2)
labels_test = dataset_raw["y_test"]
X_train_base = X_train
# 軸を交換
#X_train_base = X_train_base.transpose(1,0,2)
#labels_test_9 = labels_test
#labels_train_9 = labels_train_base

X_train_base = X_train.transpose(1,2,3,0)

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2) 
# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_train = X_train.reshape(9,48000,784)
X_validation = X_validation.reshape(9,12000,784)
X_test= X_test.reshape(9,10000,784)
#正規化 (行ごとの正規化)
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train = sp.stats.zscore(X_train,axis=1)
X_validation = sp.stats.zscore(X_validation,axis=1)
X_test = sp.stats.zscore(X_test,axis=1)
X_train = X_train.reshape(48000,28,28,9)
X_validation = X_validation.reshape(12000,28,28,9)
X_test= X_test.reshape(10000,28,28,9)

X_train = X_train.transpose(1,0,2)
X_train = [X_train[0].reshape(48000,28,28), X_train[1].reshape(48000,28,28), X_train[2].reshape(48000,28,28), 
               X_train[3].reshape(48000,28,28), X_train[4].reshape(48000,28,28), X_train[5].reshape(48000,28,28), 
               X_train[6].reshape(48000,28,28), X_train[7].reshape(48000,28,28), X_train[8].reshape(48000,28,28)]
X_train = np.array(X_train)

#X_test = X_test.transpose(1,0,2)
X_test = [X_test[0].reshape(10000,28,28), X_test[1].reshape(10000,28,28), X_test[2].reshape(10000,28,28), 
               X_test[3].reshape(10000,28,28), X_test[4].reshape(10000,28,28), X_test[5].reshape(10000,28,28), 
               X_test[6].reshape(10000,28,28), X_test[7].reshape(10000,28,28), X_test[8].reshape(10000,28,28)]
X_test = np.array(X_test)
X_test = X_test.transpose(1,2,3,0)

X_validation = X_validation.transpose(1,0,2)
X_validation = [X_validation[0].reshape(12000,28,28), X_validation[1].reshape(12000,28,28), X_validation[2].reshape(12000,28,28), 
               X_validation[3].reshape(12000,28,28), X_validation[4].reshape(12000,28,28), X_validation[5].reshape(12000,28,28), 
               X_validation[6].reshape(12000,28,28), X_validation[7].reshape(12000,28,28), X_validation[8].reshape(12000,28,28)]
X_validation = np.array(X_validation)
#X_validation = X_validation.transpose(1,2,3,0)

#X_train = X_train.transpose(3,1,2,0)
# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
#　さらにreshape""

#X_train = X_train.reshape(48000,28,28,9)
#X_validation = X_validation.reshape(12000,28,28,9)
#X_test = X_test.reshape(10000,28,28,9)
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)    

print("X_train:",X_train.shape)
print("X_validation:",X_validation.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_validation:",y_validation.shape)
print("y_test:",y_test.shape)

y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

#X_train = X_train.reshape(-1,784)
#X_validation = X_validation.reshape(-1,784)
#X_test = X_test.reshape(-1,784)

# パラメータの設定
n_features = 784
n_hidden   = 100
bias_init = 0.1

# 学習率
rate       = 0.003

# Sequentialクラスを使ってモデルを準備する
model = Sequential()
#model = multi_gpu_model(model, gpus=1)

# 隠れ層を追加
#model.add(tf.keras.Input(shape=(1,n_features,9)))

model.add(Flatten())
model.add(Dense(n_hidden,activation='relu',input_shape=(1,n_features,9)))
model.add(Dense(n_hidden,activation='relu'))
model.add(Dense(n_hidden,activation='relu'))

# 出力層を追加
model.add(Dense(10,activation='softmax'))

# 隠れ層を追加
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(28, 28, 9)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
#model.add(Flatten())
model.add(Flatten())

# 出力層を追加
model.add(Dense(10, activation='softmax'))

# TensorFlowのモデルを構築
model.compile(optimizer=tf.optimizers.Adam(rate),
              loss='categorical_crossentropy', metrics=['mae', 'accuracy'])

# Early stoppingを適用してフィッティング
log = model.fit(X_train, y_train, batch_size=200 ,epochs=1, verbose=True,
                callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                     min_delta=0, patience=10, 
                                                         verbose=1)],
                validation_data=(X_validation, y_validation))

# Test dataで予測を実行。
#pred_test = model.predict_classes(X_test)
pred_test = np.argmax(model.predict(X_test), axis=-1)

validation = (pred_test == labels_test)
size       = validation.size
size
correct    = np.count_nonzero(validation)
print(f"{correct}/{size} correct ({correct*100/size:.3f}%)")

ValueError: Found input variables with inconsistent numbers of samples: [28, 60000]

---
(MNIST保存)[https://circleken.net/2020/08/post19/]

---
(ガウシアンフィルタでぼかす)[ https://algorithm.joho.info/programming/python/opencv-gaussian-filter-py/ ]

In [4]:
with open("MNIST_pickle/raw_mnist.pkl","rb") as f:
    dataset = pickle.load(f)

In [159]:
# ガウシアンでぼかす
"""
with open("raw_mnist.pkl","rb") as f:
    dataset = pickle.load(f)

import cv2

# load image (grayscale)
# 入力画像を読み込み
img_train = dataset["X_train"]
img_test = dataset["X_test"]

#　ぼかした後の画像をまとめるnumpy配列
#　訓練データ
#dataset["X_train_val_first_gauss"] = np.array([])
first_gauss_dic = {}
list1 = [[]]*60000
list2 = [[]]*10000
for i in range(img_train.shape[0]):
    
    # convert grayscale
    # グレースケール変換
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Spatial filtering
    dst = cv2.GaussianBlur(img_train[i].reshape(28,28), ksize=(3, 3), sigmaX=0.5)

    # output
    # 結果を出力
    list1[i] = dst.reshape(-1,784)[0]

list1 = np.array(list1)
first_gauss_dic["first_gauss_train"] = list1

#　テストデータ
for i in range(img_test.shape[0]):
    
    # convert grayscale
    # グレースケール変換
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Spatial filtering
    dst = cv2.GaussianBlur(img_test[i].reshape(28,28), ksize=(3, 3), sigmaX=0.5)

    # output
    # 結果を出力
    list2[i] = dst.reshape(-1,784)[0]

list2 = np.array(list2)
first_gauss_dic["first_gauss_test"] = list2

save_file = 'first_gauss_images.pkl'
f = open(save_file,"wb")

with open(save_file, 'wb') as f:
    pickle.dump(first_gauss_dic,f, -1)"""

'\n    # output\n    # 結果を出力\n    list1[i] = dst.reshape(-1,784)[0]\n\nlist1 = np.array(list1)\nfirst_gauss_dic["first_gauss_train"] = list1\n\n#\u3000テストデータ\nfor i in range(img_test.shape[0]):\n    \n    # convert grayscale\n    # グレースケール変換\n    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)\n\n    # Spatial filtering\n    dst = cv2.GaussianBlur(img_test[i].reshape(28,28), ksize=(3, 3), sigmaX=0.5)\n\n    # output\n    # 結果を出力\n    list2[i] = dst.reshape(-1,784)[0]\n\nlist2 = np.array(list2)\nfirst_gauss_dic["first_gauss_test"] = list2\n\nsave_file = \'first_gauss_images.pkl\'\nf = open(save_file,"wb")\n\nwith open(save_file, \'wb\') as f:\n    pickle.dump(first_gauss_dic,f, -1)'

--- 
(画像を微分)[ https://algorithm.joho.info/programming/python/opencv-differential-filter-py/ ]

In [1]:
cd MNIST_pickle

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle


In [32]:
# 画像を微分する

"""
import cv2
import numpy as np
import time

list1 = [[]]*60000
list2 = [[]]*60000
list3 = [[]]*60000


# カーネル（水平、垂直方向の輪郭検出用）
kernel_x = np.array([[0, -1, 0],
                         [0, 0, 0],
                         [0, 1, 0]])

kernel_y = np.array([[0, 0, 0],
                        [-1, 0, 1],
                        [0, 0, 0]])

with open("first_gauss_images.pkl","rb") as f:
    dataset_first_gauss = pickle.load(f)

#　訓練画像を微分
for i in range(60000):
    if i%5000==0:
        print("{}まで終了".format(i))
        now = time.strftime('%Y/%m/%d %H:%M:%S')
        print(now)
    # 入力画像を読み込み
    #img = cv2.imread(osp.join(HOME, "研究/MNIST/Training_Images/First_Gaussian_Images/{}.png".format(str(i).zfill(5))))
    img = dataset_first_gauss["first_gauss_train"]
    # グレースケール変換
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 方法2
    gray_x = cv2.filter2D(img[i].reshape(28,28), cv2.CV_64F, kernel_x)
    gray_y = cv2.filter2D(img[i].reshape(28,28), cv2.CV_64F, kernel_y)
    #dst = np.sqrt(gray_x ** 2 + gray_y ** 2)
    dst = gray_x ** 2 + gray_y ** 2
    #dst = np.sqrt(np.dot(gray_x,gray_x)+ np.dot(gray_y, gray_y))
    #np.sqrt(np.linalg.norm(gray_x)+np.linalg.norm(gray_y))
    # 結果を出力
    list1[i] = gray_x.reshape(-1,784)[0]
    list2[i] = gray_y.reshape(-1,784)[0]
    list3[i] = dst.reshape(-1,784)[0]
    
list1_2 = np.array(list1)
list2_2 = np.array(list2)
list3_2 = np.array(list3)
    
diff_images["diff_x_train"] = list1_2
diff_images["diff_y_train"] = list2_2
diff_images["diff_dist_train"] = list3_2


#　テスト画像を微分
list1 = [[]]*10000
list2 = [[]]*10000
list3 = [[]]*10000

for i in range(10000):
    if i%5000==0:
        print("{}まで終了".format(i))
        now = time.strftime('%Y/%m/%d %H:%M:%S')
        print(now)
    # 入力画像を読み込み
    #img = cv2.imread(osp.join(HOME, "研究/MNIST/Training_Images/First_Gaussian_Images/{}.png".format(str(i).zfill(5))))
    img = dataset_first_gauss["first_gauss_test"]
    # グレースケール変換
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # 方法2
    gray_x = cv2.filter2D(img[i].reshape(28,28), cv2.CV_64F, kernel_x)
    gray_y = cv2.filter2D(img[i].reshape(28,28), cv2.CV_64F, kernel_y)
    #dst = np.sqrt(gray_x ** 2 + gray_y ** 2)
    dst = gray_x ** 2 + gray_y ** 2
    #dst = np.dot(gray_x,gray_x)+ np.dot(gray_y, gray_y)
    #dst = np.sqrt(np.linalg.norm(gray_x)+np.linalg.norm(gray_y))
    # 結果を出力
    list1[i] = gray_x.reshape(-1,784)[0]
    list2[i] = gray_y.reshape(-1,784)[0]
    list3[i] = dst.reshape(-1,784)[0]
    
list1_2 = np.array(list1)
list2_2 = np.array(list2)
list3_2 = np.array(list3)
    
diff_images["diff_x_test"] = list1_2
diff_images["diff_y_test"] = list2_2
diff_images["diff_dist_test"] = list3_2


save_file = 'diff_images.pkl'
f = open(save_file,"wb")

with open(save_file, 'wb') as f:
    pickle.dump(diff_images,f, -1)


0まで終了
2022/11/05 06:51:32
5000まで終了
2022/11/05 06:51:32
10000まで終了
2022/11/05 06:51:32
15000まで終了
2022/11/05 06:51:32
20000まで終了
2022/11/05 06:51:32
25000まで終了
2022/11/05 06:51:32
30000まで終了
2022/11/05 06:51:33
35000まで終了
2022/11/05 06:51:33
40000まで終了
2022/11/05 06:51:33
45000まで終了
2022/11/05 06:51:33
50000まで終了
2022/11/05 06:51:33
55000まで終了
2022/11/05 06:51:33
0まで終了
2022/11/05 06:51:33
5000まで終了
2022/11/05 06:51:33


In [3]:
cd MNIST_pickle

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle


In [4]:
for i in range(1000,1020):
    #img_x_diff = dataset_diff["diff_x_train"][i]
    #img_y_diff = dataset_diff["diff_x_train"][i]
    #img_diff_dist = dataset_diff["diff_dist_train"][i]
    #img_diff_dist = np.sqrt(img_diff_dist)
    #img_x = dataset_diff["diff_x_test"][i]
    #img_y = dataset_diff["diff_y_test"][i]
    img_dst = dataset_diff["diff_dist_train"][i]
    #print(labels_train_base[i])
    #Image.fromarray(img_x.reshape(28,28)).show()
    #Image.fromarray(img_y.reshape(28,28)).show()
    Image.fromarray(img_dst.reshape(28,28)).show()
    #Image.fromarray(img_x_diff.reshape(28,28)).show()
    #Image.fromarray(img_y_diff.reshape(28,28)).show()
    #Image.fromarray(img_diff_dist.reshape(28,28)).show()

ModuleNotFoundError: No module named 'Image'

---
画像のフーリエ記述子を求める

In [3]:
#画像のフーリエ記述子を求める

import pickle
from math import pi

# epsilon
ep = 10 ** -12

# Q = 4とする
Q = 4
#list_name = ["train_four_coeff_0","train_four_coeff_1","train_four_coeff_2","train_four_coeff_3","train_four_coeff_4"]
list_name = [{},{},{},{},{}]
with open("diff_images.pkl","rb") as f:
    dataset_diff = pickle.load(f)
diff_x_train = dataset_diff["diff_x_train"]
diff_y_train = dataset_diff["diff_y_train"]
diff_x_test = dataset_diff["diff_x_test"]
diff_y_test = dataset_diff["diff_y_test"]
diff_train_image = diff_x_train - diff_y_train*1j
diff_test_image = diff_x_test - diff_y_test*1j
sqrt_train = (dataset_diff["diff_dist_train"]+ep) ** 0.5
sqrt_test = (dataset_diff["diff_dist_test"]+ep) ** 0.5
train_four_coeff_0 = {}
train_four_coeff_1 = {}
train_four_coeff_2 = {}
train_four_coeff_3 = {}
train_four_coeff_4 = {}
test_four_coeff = {}

#　フーリエ記述子導出
for i in range(Q+1):
    dq_train = sqrt_train/(2*pi) * (diff_train_image/sqrt_train) ** i
    dq_test = sqrt_test/(2*pi) * (diff_test_image/sqrt_test) ** i
    train_four_coeff_real = dq_train.real
    train_four_coeff_imag = dq_train.imag
    test_four_coeff_real = dq_test.real
    test_four_coeff_imag = dq_test.imag
    if i==0:
        list_name[i]["train_four_coeff_layer_{}_real".format(i)] = train_four_coeff_real
        test_four_coeff["test_coeff_layer_{}_real".format(i)] =  test_four_coeff_real
    else:
        list_name[i]["train_four_coeff_layer_{}_real".format(i)] = train_four_coeff_real
        list_name[i]["train_four_coeff_layer_{}_imag".format(i)] = train_four_coeff_imag
        test_four_coeff["test_coeff_layer_{}_real".format(i)] =  test_four_coeff_real
        test_four_coeff["test_coeff_layer_{}_imag".format(i)] =  test_four_coeff_imag

In [13]:
#　画像のshape確認
print(test_four_coeff.keys())
print(test_four_coeff["test_coeff_layer_0_real"].shape)
print(test_four_coeff["test_coeff_layer_1_real"].shape)
print(test_four_coeff["test_coeff_layer_1_imag"].shape)
print(test_four_coeff["test_coeff_layer_2_real"].shape)
print(test_four_coeff["test_coeff_layer_2_imag"].shape)
print(test_four_coeff["test_coeff_layer_3_real"].shape)
print(test_four_coeff["test_coeff_layer_3_imag"].shape)
print(test_four_coeff["test_coeff_layer_4_real"].shape)
print(test_four_coeff["test_coeff_layer_4_imag"].shape)

dict_keys(['test_coeff_layer_0_real', 'test_coeff_layer_1_real', 'test_coeff_layer_1_imag', 'test_coeff_layer_2_real', 'test_coeff_layer_2_imag', 'test_coeff_layer_3_real', 'test_coeff_layer_3_imag', 'test_coeff_layer_4_real', 'test_coeff_layer_4_imag'])
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)
(10000, 784)


In [4]:
#　test保存
"""
save_file_test_1 = 'test_four_coeff_layer_0_to_2_ver2.pkl'
save_file_test_2 = 'test_four_coeff_layer_3_to_4_ver2.pkl'

with open(save_file_test_1, 'wb') as f:
    test_layer_0_to_2 = {}
    test_layer_0_to_2["test_coeff_layer_0_real"] = test_four_coeff["test_coeff_layer_0_real"]
    test_layer_0_to_2["test_coeff_layer_1_real"] = test_four_coeff["test_coeff_layer_1_real"]
    test_layer_0_to_2["test_coeff_layer_1_imag"] = test_four_coeff["test_coeff_layer_1_imag"]
    test_layer_0_to_2["test_coeff_layer_2_real"] = test_four_coeff["test_coeff_layer_2_real"]
    test_layer_0_to_2["test_coeff_layer_2_imag"] = test_four_coeff["test_coeff_layer_2_imag"]
    pickle.dump(test_layer_0_to_2,f, -1)

del test_layer_0_to_2

with open(save_file_test_2, 'wb') as f:
    test_layer_3_to_4 = {}
    test_layer_3_to_4["test_coeff_layer_3_real"] = test_four_coeff["test_coeff_layer_3_real"]
    test_layer_3_to_4["test_coeff_layer_3_imag"] = test_four_coeff["test_coeff_layer_3_imag"]
    test_layer_3_to_4["test_coeff_layer_4_real"] = test_four_coeff["test_coeff_layer_4_real"]
    test_layer_3_to_4["test_coeff_layer_4_imag"] = test_four_coeff["test_coeff_layer_4_imag"]
    pickle.dump(test_layer_3_to_4,f, -1)"""

In [12]:
#　フーリエ記述子を保存
del train_four_coeff_3_to_4
del test_four_coeff

save_file = 'train_four_coeff_layer_0_to_2_ver2.pkl'

with open(save_file, 'wb') as f:
    pickle.dump(train_four_coeff_0_to_2,f, -1)

    
"""save_file = 'diff_images.pkl'
f = open(save_file,"wb")

with open(save_file, 'wb') as f:
    pickle.dump(diff_images,f, -1) """   
    

"""save_file_2 = ('second_MNIST/train_four_coeff_layer_3_to_4_ver2.pkl')
with open(save_file_2, 'wb') as f:
    pickle.dump(train_four_coeff_3_to_4,f, -1)
    
save_file_3 = ('second_MNIST/test_four_coeff_layer_ver2.pkl')
with open(save_file_3, 'wb') as f:
    pickle.dump(test_four_coeff,f, -1)"""

OSError: [Errno 5] Input/output error

In [3]:
"""
# epsilon
e = 10 ** -12

# Q = 4とする
Q = 4

import time
#four_coeff = {}
#four_list = [[]]
#four_list = [[]]*784
#four_list = [four_list]*9

with open("diff_images.pkl","rb") as f:
    dataset_diff = pickle.load(f)


#　訓練データ
for i in range(1,2*Q+1):
    four_coeff = {}
    for j in range(60000):
        diff_x = dataset_diff["diff_x_train"][j].reshape(28,28)
        diff_y = dataset_diff["diff_y_train"][j].reshape(28,28)
        sqrt = dataset_diff["diff_dist_train"][j].reshape(28,28)
        #kari_sqrt =  np.sqrt(sqrt[i]**2+e)
        #kari_sqrt =  np.sqrt(np.linalg.matrix_power(sqrt,2)+e)
        kari_sqrt =  np.sqrt(sqrt+e)
        dq_1 = kari_sqrt/(2*pi)
        #dq_2 = (diff_x - diff_y)/kari_sqrt
        imag_diff_y = diff_y*1j
        dq_2 = diff_x - imag_diff_y
        #メモリ解放
        #del diff_x, diff_y, sqrt, kari_sqrt

        #dq = dq_1 * np.linalg.matrix_power(dq_2,j-1)
        if i==0:
            dq = dq_1
        elif i==1:
            dq = (dq_2)/(2*pi)
        else:
            dq = np.linalg.matrix_power(dq_2,i)/(np.linalg.matrix_power(kari_sqrt,i-1)*2*pi)
        if j==0:
            kari_array = np.array([dq.reshape(-1,784)[0]])
        else:
            kari_array = np.vstack((kari_array, dq.reshape(-1,784)[0]))
    
    four_coeff["train_four_coeff_layer{}".format(str(i))] = kari_array
    
    print("{}まで終了".format(i))
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    
    save_file = ('train_four_coeff_layer{}.pkl'.format(str(i)))
    with open(save_file, 'wb') as f:
        pickle.dump(four_coeff,f, -1)



# テストデータ
for i in range(0,2*Q+1):
    four_coeff = {}
    for j in range(10000):
        diff_x = dataset_diff["diff_x_test"][j].reshape(28,28)
        diff_y = dataset_diff["diff_y_test"][j].reshape(28,28)
        sqrt = dataset_diff["diff_dist_test"][j].reshape(28,28)
        #kari_sqrt =  np.sqrt(sqrt[i]**2+e)
        kari_sqrt = np.sqrt(sqrt+e)
        dq_1 = kari_sqrt/(2*pi)
        #dq_2 = (diff_x - diff_y)/kari_sqrt
        imag_diff_y = diff_y*1j
        dq_2 = diff_x - imag_diff_y
        
        #メモリ解放
        #del diff_x, diff_y, sqrt, kari_sqrt

        #dq = dq_1 * np.linalg.matrix_power(dq_2,j-1)
        #dq = dq_1 * (dq_2**(i-1))
        if i==0:
            dq = dq_1
        elif i==1:
            dq = (dq_2)/(2*pi)
        else:
            dq = np.linalg.matrix_power(dq_2,i)/(np.linalg.matrix_power(kari_sqrt,i-1)*2*pi)
        if j==0:
            kari_array = np.array([dq.reshape(-1,784)[0]])
        else:
            kari_array = np.vstack((kari_array, dq.reshape(-1,784)[0]))
    
    four_coeff["test_four_coeff_layer{}".format(str(i))] = kari_array
    
    print("{}まで終了".format(i))
    now = time.strftime('%Y/%m/%d %H:%M:%S')
    print(now)
    
    save_file = ('test_four_coeff_layer{}.pkl'.format(str(i)))
    with open(save_file, 'wb') as f:
        pickle.dump(four_coeff,f, -1)
"""

1まで終了
2022/11/05 14:22:21
2まで終了
2022/11/05 15:03:10
3まで終了
2022/11/05 15:43:53
4まで終了
2022/11/05 16:24:57
5まで終了
2022/11/05 17:05:48
6まで終了
2022/11/05 17:46:29
7まで終了
2022/11/05 18:27:31
8まで終了
2022/11/05 19:08:34
0まで終了
2022/11/05 19:09:24
1まで終了
2022/11/05 19:10:36
2まで終了
2022/11/05 19:11:52
3まで終了
2022/11/05 19:13:06
4まで終了
2022/11/05 19:14:22
5まで終了
2022/11/05 19:15:38
6まで終了
2022/11/05 19:16:55
7まで終了
2022/11/05 19:18:11
8まで終了
2022/11/05 19:19:27


---
2回目のガウスぼかし

In [5]:
cd MNIST_pickle/second_MNIST

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle/second_MNIST


In [7]:
#　データの前処理
import pickle
#　訓練データ
with open("train_four_coeff_layer0_real_ver2.pkl","rb") as f:
    dataset_train_0_real = pickle.load(f)
with open("train_four_coeff_layer1_real_ver2.pkl","rb") as f:
    dataset_train_1_real = pickle.load(f)
with open("train_four_coeff_layer1_imag_ver2.pkl","rb") as f:
    dataset_train_1_imag = pickle.load(f)
with open("train_four_coeff_layer2_real_ver2.pkl","rb") as f:
    dataset_train_2_real = pickle.load(f)
with open("train_four_coeff_layer2_imag_ver2.pkl","rb") as f:
    dataset_train_2_imag = pickle.load(f)
with open("train_four_coeff_layer3_real_ver2.pkl","rb") as f:
    dataset_train_3_real = pickle.load(f)
with open("train_four_coeff_layer3_imag_ver2.pkl","rb") as f:
    dataset_train_3_imag = pickle.load(f)
with open("train_four_coeff_layer4_real_ver2.pkl","rb") as f:
    dataset_train_4_real = pickle.load(f)
with open("train_four_coeff_layer4_imag_ver2.pkl","rb") as f:
    dataset_train_4_imag = pickle.load(f)

In [6]:
#　テストデータの前処理
with open("test_four_coeff_layer_0_to_2_ver2.pkl","rb") as f:
    dataset_test_0_to_2 = pickle.load(f)
with open("test_four_coeff_layer_3_to_4_ver2.pkl","rb") as f:
    dataset_test_3_to_4 = pickle.load(f)

In [127]:
dataset_test_3_to_4.keys()

dict_keys(['test_coeff_layer_3_real', 'test_coeff_layer_3_imag', 'test_coeff_layer_4_real', 'test_coeff_layer_4_imag'])

In [8]:
list_train_four_coeff = np.stack((dataset_train_0_real["train_four_coeff_layer_0_real"],dataset_train_1_real["train_four_coeff_layer_1_real"],
                        dataset_train_1_imag["train_four_coeff_layer_1_imag"],dataset_train_2_real["train_four_coeff_layer_2_real"],
                        dataset_train_2_imag["train_four_coeff_layer_2_imag"],dataset_train_3_real["train_four_coeff_layer_3_real"],
                        dataset_train_3_imag["train_four_coeff_layer_3_imag"],dataset_train_4_real["train_four_coeff_layer_4_real"],
                        dataset_train_4_imag["train_four_coeff_layer_4_imag"]),axis=0)
list_test_four_coeff = np.stack((dataset_test_0_to_2["test_coeff_layer_0_real"],dataset_test_0_to_2["test_coeff_layer_1_real"],
                                dataset_test_0_to_2["test_coeff_layer_1_imag"],dataset_test_0_to_2["test_coeff_layer_2_real"],
                                dataset_test_0_to_2["test_coeff_layer_2_imag"],dataset_test_3_to_4["test_coeff_layer_3_real"],
                                 dataset_test_3_to_4["test_coeff_layer_3_imag"],dataset_test_3_to_4["test_coeff_layer_4_real"],
                                 dataset_test_3_to_4["test_coeff_layer_4_imag"]),axis=0)
#x_header = np.array(pd.DataFrame(list_train_four_coeff[0]).columns.tolist())
#np.savez('mnist_train_four_coeff', x_header, list_train_four_coeff)
"""#　辞書型にする
train_layer = {}
train_layer["train_0_real"] = dataset_train_0_real["train_four_coeff_layer_0_real"]
train_layer["train_1_real"] = dataset_train_1_real["train_four_coeff_layer_1_real"]
train_layer["train_1_imag"] = dataset_train_1_imag["train_four_coeff_layer_1_imag"]
train_layer["train_2_real"] = dataset_train_2_real["train_four_coeff_layer_2_real"]
train_layer["train_2_imag"] = dataset_train_2_imag["train_four_coeff_layer_2_imag"]
train_layer["train_3_real"] = dataset_train_3_real["train_four_coeff_layer_3_real"]
train_layer["train_3_imag"] = dataset_train_3_imag["train_four_coeff_layer_3_imag"]
train_layer["train_4_real"] = dataset_train_4_real["train_four_coeff_layer_4_real"]
train_layer["train_4_imag"] = dataset_train_4_imag["train_four_coeff_layer_4_imag"]"""

'#\u3000辞書型にする\ntrain_layer = {}\ntrain_layer["train_0_real"] = dataset_train_0_real["train_four_coeff_layer_0_real"]\ntrain_layer["train_1_real"] = dataset_train_1_real["train_four_coeff_layer_1_real"]\ntrain_layer["train_1_imag"] = dataset_train_1_imag["train_four_coeff_layer_1_imag"]\ntrain_layer["train_2_real"] = dataset_train_2_real["train_four_coeff_layer_2_real"]\ntrain_layer["train_2_imag"] = dataset_train_2_imag["train_four_coeff_layer_2_imag"]\ntrain_layer["train_3_real"] = dataset_train_3_real["train_four_coeff_layer_3_real"]\ntrain_layer["train_3_imag"] = dataset_train_3_imag["train_four_coeff_layer_3_imag"]\ntrain_layer["train_4_real"] = dataset_train_4_real["train_four_coeff_layer_4_real"]\ntrain_layer["train_4_imag"] = dataset_train_4_imag["train_four_coeff_layer_4_imag"]'

In [289]:
# ガウシアンでぼかす
import cv2

# load image (grayscale)
# 入力画像を読み込み
#img_train = dataset["X_train"]
#img_test = dataset["X_test"]

#　ぼかした後の画像をまとめるnumpy配列
#　訓練データ
#dataset["X_train_val_first_gauss"] = np.array([])
#second_gauss_train_0_to_2 = {}
#second_gauss_train_3_to_4 = {}

Q = 4
sigma_value = 1
second_gauss_train = np.zeros((9,60000,784))
second_gauss_test = np.zeros((9,10000,784))

for i in range(2*Q+1):
    second_gauss_train[i] = cv2.GaussianBlur(list_train_four_coeff[i], ksize=(3, 3), sigmaX=sigma_value)
    second_gauss_test[i] = cv2.GaussianBlur(list_test_four_coeff[i], ksize=(3, 3), sigmaX=sigma_value)
    
#　いらないかも?
    """list_train_0_real = cv2.GaussianBlur(train_layer["train_0_real"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_1_real = cv2.GaussianBlur(train_layer["train_1_real"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_1_imag = cv2.GaussianBlur(train_layer["train_1_imag"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_2_real = cv2.GaussianBlur(train_layer["train_2_real"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_2_imag = cv2.GaussianBlur(train_layer["train_2_imag"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_3_real = cv2.GaussianBlur(train_layer["train_3_real"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_3_imag = cv2.GaussianBlur(train_layer["train_3_imag"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_4_real = cv2.GaussianBlur(train_layer["train_4_real"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)
    list_train_4_imag = cv2.GaussianBlur(train_layer["train_4_imag"].reshape(60000,28,28), ksize=(3, 3), sigmaX=sigma_value)

    # 結果を保存
    diff_test_x_header = np.array(pd.DataFrame(diff_X_test).columns.tolist())
    diff_test_y_header = np.array(pd.DataFrame(diff_y_test).columns.tolist())
    diff_test_dst_header = np.array(pd.DataFrame(dst).columns.tolist())
    np.savez('fashion_MNIST/X_diff_test', diff_test_x_header, diff_X_test)
    np.savez('fashion_MNIST/y_diff_test', diff_test_y_header, diff_y_test)
    np.savez('fashion_MNIST/dst_test', diff_test_dst_header, dst)"""

    """
    # 保存
    save_file_1 = 'second_gauss_train_0_real_ver2.pkl'
    save_file_2 = 'second_gauss_train_1_real_ver2.pkl'
    save_file_3 = 'second_gauss_train_1_imag_ver2.pkl'
    save_file_4 = 'second_gauss_train_2_real_ver2.pkl'
    save_file_5 = 'second_gauss_train_2_imag_ver2.pkl'
    save_file_6 = 'second_gauss_train_3_real_ver2.pkl'
    save_file_7 = 'second_gauss_train_3_imag_ver2.pkl'
    save_file_8 = 'second_gauss_train_4_real_ver2.pkl'
    save_file_9 = 'second_gauss_train_4_imag_ver2.pkl'

    #　保存
    f1 = open(save_file_1,"wb")
    with open(save_file_1, 'wb') as f1:
        pickle.dump(list_train_0_real,f1, -1)
    ###
    f2 = open(save_file_2,"wb")
    with open(save_file_2, 'wb') as f2:
        pickle.dump(list_train_1_real,f2, -1)
    ### 
    f3 = open(save_file_3,"wb")
    with open(save_file_3, 'wb') as f3:
        pickle.dump(list_train_1_imag,f3, -1)
    del list_train_1_imag
    ###
    f4 = open(save_file_4,"wb")
    with open(save_file_4, 'wb') as f4:
        pickle.dump(list_train_2_real,f4, -1)
    del list_train_2_real
    ###
    f5 = open(save_file_5,"wb")
    with open(save_file_5, 'wb') as f5:
        pickle.dump(list_train_2_imag,f5, -1)
    del list_train_2_imag
    ###
    f6 = open(save_file_6,"wb")
    with open(save_file_6, 'wb') as f6:
        pickle.dump(list_train_3_real,f6, -1)
    del list_train_3_real
    ###
    f7 = open(save_file_7,"wb")
    with open(save_file_7, 'wb') as f7:
        pickle.dump(list_train_3_imag,f7, -1)
    del list_train_3_imag
    ###
    f8 = open(save_file_8,"wb")
    with open(save_file_8, 'wb') as f8:
        pickle.dump(list_train_4_real,f8, -1)
    del list_train_4_real
    ###
    f9 = open(save_file_9,"wb")
    with open(save_file_9, 'wb') as f9:
        pickle.dump(list_train_4_imag,f9, -1)
    del list_train_4_imag
    """

In [178]:
second_gauss_train.shape

(9, 60000, 784)

In [30]:
#　テストデータ
"""

with open("test_four_coeff_layer_0_to_2_ver2.pkl","rb") as f:
    dataset_test_0_to_2 = pickle.load(f)
with open("test_four_coeff_layer_3_to_4_ver2.pkl","rb") as f:
    dataset_test_3_to_4 = pickle.load(f)
    

#dataset["X_train_val_first_gauss"] = np.array([])
second_gauss_test_0_to_2 = {}
second_gauss_test_3_to_4 = {}

import cv2
#　ガウシアンぼかし
second_gauss_test_0_to_2["test_0_real"] = cv2.GaussianBlur(dataset_test_0_to_2["test_coeff_layer_0_real"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_0_to_2["test_1_real"] = cv2.GaussianBlur(dataset_test_0_to_2["test_coeff_layer_1_real"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_0_to_2["test_1_imag"] = cv2.GaussianBlur(dataset_test_0_to_2["test_coeff_layer_1_imag"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_0_to_2["test_2_real"] = cv2.GaussianBlur(dataset_test_0_to_2["test_coeff_layer_2_real"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_0_to_2["test_2_imag"] = cv2.GaussianBlur(dataset_test_0_to_2["test_coeff_layer_2_imag"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_3_to_4["test_3_real"] = cv2.GaussianBlur(dataset_test_3_to_4["test_coeff_layer_3_real"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_3_to_4["test_3_imag"] = cv2.GaussianBlur(dataset_test_3_to_4["test_coeff_layer_3_imag"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_3_to_4["test_4_real"] = cv2.GaussianBlur(dataset_test_3_to_4["test_coeff_layer_4_real"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)
second_gauss_test_3_to_4["test_4_imag"] = cv2.GaussianBlur(dataset_test_3_to_4["test_coeff_layer_4_imag"].reshape(10000,28,28), ksize=(3, 3), sigmaX=sigma_value)

# 保存
save_file = 'second_gauss_test_0_to_2_ver2.pkl'
with open(save_file, 'wb') as f:
    pickle.dump(second_gauss_test_0_to_2,f, -1)
    
save_file = 'second_gauss_test_3_to_4_ver2.pkl'
with open(save_file, 'wb') as f:
    pickle.dump(second_gauss_test_3_to_4,f, -1)"""

---

In [31]:
#img = second_gauss_test[0][4].reshape(28,28)*128
k = 1000
for i in range(k,k+10):
    img = cv2.GaussianBlur(list_train_four_coeff[0][i].reshape(28,28), ksize=(3, 3), sigmaX=10)*4
    Image.fromarray(img).show()
#img_gaussian = cv2.imread(osp.join(HOME, "研究/MNIST/Training_Images/First_Gaussian_Images/{}.png".format(str(3).zfill(5))),cv2.COLOR_BGR2RGB)
#cv2.imshow("a",img)

#img_np = np.array(img)
#img_gaussian_np = np.array(img_gaussian)

#img_resize = cv2.resize(img, (28,28))
#img_gaussian_resize = cv2.resize(img_gaussian, (256,256))
#img_np = np.array(img_resize)
#img_gaussian_np = np.array(img_gaussian_resize)
#Image.fromarray(img).show()
#Image.fromarray(img_gaussian_resize).show()

In [231]:
list_train_four_coeff.shape

(9, 60000, 784)

In [49]:
img_resize.shape

(28, 28)

In [156]:
"""for i in range(28):
    for j in range(28):
        #print("普通の画像")
        if (img[i][j]<10):
            print("    ",end="")
            print(img[i][j], end="")
        elif (img[i][j]<100):
            print("  ",end="")
            print(img[i][j],end="")
        else:
            print(" ",end="")
            print(img[i][j],end="")
    print("")
print("")

for i in range(28):
    for j in range(28):
        #print("普通の画像")
        if (img_gaussian[i][j]<10):
            print("    ",end="")
            print(img_gaussian[i][j], end="")
        elif (img_gaussian[i][j]<100):
            print("  ",end="")
            print(img_gaussian[i][j],end="")
        else:
            print(" ",end="")
            print(img_gaussian[i][j],end="")
    print("")
print(img.shape)
print(img_gaussian.shape)
"""

    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   

'print(img.shape)\nprint(img_gaussian.shape)'

In [2]:
cd MNIST_pickle

/home/yy/y.shinozaki/研究/MNIST/MNIST_pickle


In [7]:
# MNIST 複数チャンネル用(Pytorch)

from tensorflow.keras.layers import Flatten
import pickle
from sklearn.model_selection import train_test_split
import scipy as sp
from tensorflow.keras.utils import to_categorical
import torch

with open("second_gauss_train_0_to_2.pkl","rb") as f:
    dataset_train_0_to_2 = pickle.load(f)
with open("second_gauss_train_3_to_4.pkl","rb") as f:
    dataset_train_3_to_4 = pickle.load(f)
with open("second_gauss_test.pkl","rb") as f:
    dataset_test = pickle.load(f)
with open("raw_mnist.pkl","rb") as f:
    dataset_raw = pickle.load(f)

# Kerasに付属の手書き数字画像データをダウンロード
labels_train_base = dataset_raw["y_train"]

#X_train_base = dataset_train_0_to_2["second_gauss_train_0_to_2"][2]
X_train_base = np.vstack((dataset_train_0_to_2["second_gauss_train_0_to_2"],dataset_train_3_to_4["second_gauss_train_3_to_4"]))
#X_train_base = X_train_base      
    
X_test = dataset_test["second_gauss_test"]
X_test = X_test.reshape(9,10000,784)
X_test = X_test.transpose(1,0,2)
labels_test = dataset_raw["y_test"]
# 軸を交換
X_train_base = X_train_base.transpose(1,0,2)
labels_test_9 = labels_test
labels_train_9 = labels_train_base
#X_train_loader = torch.utils.data.DataLoader(dataset=X_train_base, batch_size=10000, shuffle=True)
# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2) 
# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き

#正規化 (行ごとの正規化)
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train = sp.stats.zscore(X_train,axis=1)
X_validation = sp.stats.zscore(X_validation,axis=1)
X_test = sp.stats.zscore(X_test,axis=1)
# labels_train, labels_validation, labels_test をダミー変数化して y_train, y_validation, y_test に格納する
y_train = to_categorical(labels_train)
y_validation = to_categorical(labels_validation)
y_test = to_categorical(labels_test)

# dataloader作成
X_train_loader = torch.utils.data.DataLoader(dataset=X_train, batch_size=100, shuffle=False)
X_validation_loader = torch.utils.data.DataLoader(dataset=X_validation, batch_size=100, shuffle=False)
X_test_loader = torch.utils.data.DataLoader(dataset=X_test, batch_size=100, shuffle=False)
y_train_loader = torch.utils.data.DataLoader(dataset=y_train, batch_size=100, shuffle=False)
y_validation_loader = torch.utils.data.DataLoader(dataset=y_validation, batch_size=100, shuffle=False)
y_test_loader = torch.utils.data.DataLoader(dataset=y_test, batch_size=100, shuffle=False)

In [12]:
print("X_train:",X_train.shape)
print("X_tvalidation:",X_validation.shape)
print("X_test:",X_test.shape)
print("y_train:",y_train.shape)
print("y_validation:",y_validation.shape)
print("y_test:",y_test.shape)

X_train: (48000, 9, 784)
X_tvalidation: (12000, 9, 784)
X_test: (10000, 9, 784)
y_train: (48000, 10)
y_validation: (12000, 10)
y_test: (10000, 10)


In [27]:
# Keras　ニューラルネットワーク
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from datetime import datetime
from zoneinfo import ZoneInfo

#----------------------------------------------------------
# ハイパーパラメータなどの設定値
num_epochs = 10         # 学習を繰り返す回数
num_batch = 100         # 一度に処理する画像の枚数
learning_rate = 0.001   # 学習率
image_size = 28*28      # 画像の画素数(幅x高さ)

# GPU(CUDA)が使えるかどうか？
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#----------------------------------------------------------
# 学習用／評価用のデータセットの作成

# 変換方法の指定
transform = transforms.Compose([
    transforms.ToTensor()
    ])

# MNISTデータの取得
# https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST
# 学習用
train_dataset = datasets.MNIST(
    './data',               # データの保存先
    train = True,           # 学習用データを取得する
    download = True,        # データが無い時にダウンロードする
    transform = transform   # テンソルへの変換など
    )
# 評価用
test_dataset = datasets.MNIST(
    './data', 
    train = False,
    transform = transform
    )


# データローダー
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = num_batch,
    shuffle = True)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,     
    batch_size = num_batch,
    shuffle = True)

#----------------------------------------------------------
# ニューラルネットワークモデルの定義
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()

        # 各クラスのインスタンス（入出力サイズなどの設定）
        self.fc1 = nn.Linear(input_size, 100)
        self.fc2 = nn.Linear(100, output_size)
        #self.fc3 = nn.Linear(output_size, 3)

    def forward(self, x):
        # 順伝播の設定（インスタンスしたクラスの特殊メソッド(__call__)を実行）
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#----------------------------------------------------------
# ニューラルネットワークの生成
model = Net(image_size, 10).to(device)

#----------------------------------------------------------
# 損失関数の設定
criterion = nn.CrossEntropyLoss() 

#----------------------------------------------------------
# 最適化手法の設定
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) 

#----------------------------------------------------------
# 学習
model.train()  # モデルを訓練モードにする

for epoch in range(num_epochs): # 学習を繰り返し行う
    loss_sum = 0

    #for inputs, labels in X_train_loader, y_train_loader:
    for inputs, labels in train_dataloader:
        
        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizerを初期化
        optimizer.zero_grad()

        # ニューラルネットワークの処理を行う
        inputs = inputs.view(-1, image_size) # 画像データ部分を一次元へ並び変える
        outputs = model(inputs)

        # 損失(出力とラベルとの誤差)の計算
        loss = criterion(outputs, labels)
        loss_sum += loss

        # 勾配の計算
        loss.backward()

        # 重みの更新
        optimizer.step()

    #　現在時刻の取得
    #now = time.strftime("%Y%m%d %H:%M:%S")
    now = datetime.now(ZoneInfo("Asia/Tokyo"))
    
    # 学習状況の表示
    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss_sum.item() / len(train_dataloader)}", end=" ")
    print(",Time",now)
    # モデルの重みの保存
    torch.save(model.state_dict(), 'model_weights.pth')

#----------------------------------------------------------
# 評価
model.eval()  # モデルを評価モードにする

loss_sum = 0
correct = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:

        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)
        labels = labels.to(device)

        # ニューラルネットワークの処理を行う
        inputs = inputs.view(-1, image_size) # 画像データ部分を一次元へ並び変える
        outputs = model(inputs)

        # 損失(出力とラベルとの誤差)の計算
        loss_sum += criterion(outputs, labels)

        # 正解の値を取得
        pred = outputs.argmax(1)
        # 正解数をカウント
        correct += pred.eq(labels.view_as(pred)).sum().item()

print(f"Loss: {loss_sum.item() / len(test_dataloader)}, Accuracy: {100*correct/len(test_dataset)}% ({correct}/{len(test_dataset)})")

Epoch: 1/10, Loss: 0.662894287109375 ,Time 2022-11-14 12:05:11.590052+09:00
Epoch: 2/10, Loss: 0.2668366495768229 ,Time 2022-11-14 12:05:15.259570+09:00
Epoch: 3/10, Loss: 0.20937152862548827 ,Time 2022-11-14 12:05:19.162526+09:00
Epoch: 4/10, Loss: 0.17454419453938802 ,Time 2022-11-14 12:05:22.735976+09:00
Epoch: 5/10, Loss: 0.14961835225423178 ,Time 2022-11-14 12:05:26.296329+09:00
Epoch: 6/10, Loss: 0.1300244394938151 ,Time 2022-11-14 12:05:29.960918+09:00
Epoch: 7/10, Loss: 0.11467575073242188 ,Time 2022-11-14 12:05:33.535177+09:00
Epoch: 8/10, Loss: 0.10197804768880209 ,Time 2022-11-14 12:05:37.130389+09:00
Epoch: 9/10, Loss: 0.09095195770263671 ,Time 2022-11-14 12:05:40.677206+09:00
Epoch: 10/10, Loss: 0.08188023885091146 ,Time 2022-11-14 12:05:44.360704+09:00
Loss: 0.10393674850463867, Accuracy: 96.83% (9683/10000)


---
(MNIST)[ https://exture-ri.com/2021/01/11/pytorch-cnn/ ]

In [95]:
#ライブラリの準備
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

'''Datasetの準備'''
train_dataset = MNIST(root='mydata', train=True,  transform=transforms.ToTensor(), download=True)
test_dataset  = MNIST(root='mydata', train=False, transform=transforms.ToTensor())

'''DataLoaderを作成'''
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader  = DataLoader(dataset=test_dataset,  batch_size=100, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mydata/MNIST/raw/train-images-idx3-ubyte.gz to mydata/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mydata/MNIST/raw/train-labels-idx1-ubyte.gz to mydata/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mydata/MNIST/raw/t10k-images-idx3-ubyte.gz to mydata/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mydata/MNIST/raw/t10k-labels-idx1-ubyte.gz to mydata/MNIST/raw



In [96]:
# モデルと最適化手法の定義の定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7 * 7 * 32, 10)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [97]:
#訓練用の関数を定義
def train(train_loader):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss / len(train_loader) 
    return train_loss

'''評価用の関数を定義'''
def valid(test_loader):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            predicted = outputs.max(1, keepdim=True)[1]
            labels = labels.view_as(predicted)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

In [99]:
#誤差(loss)を記録する空の配列を用意
loss_list = []
val_loss_list = []
val_acc_list = []

'''学習'''
for epoch in range(10):
    loss = train(train_loader)
    val_loss, val_acc = valid(test_loader)
    print('epoch %d, loss: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, loss, val_loss, val_acc))
    loss_list.append(loss)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)


NameError: name 'test_acc' is not defined